# Atelier 1: Introduction aux LLMs et aux SLMs

## Table des matières:

1. Introduction aux LLMs et SLMs
2. Utilisation basique d'un LLM
3. Utilisation d'un SLM
4. Conclusion et ressources supplémentaires

---

Bienvenue à cet atelier sur les modèles de langage! Aujourd'hui, nous allons explorer les différences entre les grands modèles de langage (LLMs) et les petits modèles de langage (SLMs), et apprendre à les utiliser dans des applications pratiques.

**Objectifs de l'atelier:**
- Comprendre les différences entre LLMs et SLMs
- Apprendre à interagir avec un SLM local
- Créer des prompts efficaces
- Explorer les paramètres qui influencent les réponses

## 1. Introduction aux LLMs et SLMs

### Qu'est-ce qu'un LLM (Large Language Model)?
- Modèle de langage de grande taille (des dizaines ou centaines de milliards de paramètres)
- Exemples: GPT-4, Claude, PaLM, Llama 2 (70B)
- Nécessite généralement une infrastructure cloud pour fonctionner
- Performances élevées sur une large gamme de tâches

### Qu'est-ce qu'un SLM (Small Language Model)?
- Modèle de langage de taille réduite (moins de 13 milliards de paramètres)
- Exemples: Mistral 7B (Français), LiquidAI, Phi-2, Llama 2 (7B)
- Peut fonctionner sur un ordinateur personnel
- Performances plus limitées mais suffisantes pour de nombreuses applications

### Avantages des SLMs:
- Confidentialité des données (exécution locale)
- Pas de coûts d'API
- Personnalisation possible
- Utilisation hors ligne
- Contrôle total sur les paramètres

### Limitations des SLMs:
- Connaissances plus limitées
- Capacités de raisonnement réduites
- Hallucinations plus fréquentes
- Contexte plus court

## 2. Configuration de l'environnement

Dans cet atelier, nous utilisons:
- Un serveur local llama.cpp qui expose une API compatible avec OpenAI
- La bibliothèque Python `openai` pour communiquer avec ce serveur

Le serveur llama.cpp devrait déjà être en cours d'exécution sur le port 8080 avec le modèle que nous avons téléchargé.

Commençons par installer et configurer la bibliothèque OpenAI pour communiquer avec notre serveur local:

In [ ]:
from openai import OpenAI
import requests
import json
import time

# Configuration pour utiliser le serveur llama.cpp local
client = OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="sk-no-key-required"  # Clé factice, llama.cpp n'en a pas besoin
)

# Fonction pour vérifier que le serveur est en cours d'exécution
def check_server():
    try:
        base_url_str = str(client.base_url)
        
        # Supprimer "/v1" de l'URL de base pour accéder à "/models"
        if base_url_str.endswith("/v1/"):
            base_url_str = base_url_str[:-3]
            
        response = requests.get(f"{base_url_str}/models")
        if response.status_code == 200:
            print("✅ Connexion au serveur llama.cpp réussie!")
            return True
        else:
            print(f"❌ Le serveur a répondu avec le code {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Impossible de se connecter au serveur llama.cpp.")
        print("Assurez-vous que le serveur est en cours d'exécution sur http://localhost:8080")
        return False

# Vérification de la connexion au serveur
check_server()

## 3. Utilisation basique d'un SLM

Maintenant que nous sommes connectés au serveur, nous pouvons commencer à interagir avec notre modèle de langage. Commençons par créer une fonction simple pour envoyer des requêtes au modèle:

In [ ]:
def query_llm(prompt, temperature=0.7, max_tokens=256):
    """
    Envoie une requête au modèle de langage et retourne sa réponse.
    
    Args:
        prompt (str): Le texte d'entrée pour le modèle
        temperature (float): Contrôle la créativité (0.0 = déterministe, 1.0 = créatif)
        max_tokens (int): Nombre maximum de tokens à générer
        
    Returns:
        str: La réponse générée par le modèle
    """
    try:
        response = client.completions.create(
            model="local-model",  # Le nom du modèle n'est pas important pour llama.cpp
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Erreur lors de la requête au modèle: {str(e)}"

# Test simple
prompt = "Explique ce qu'est l'intelligence artificielle en 3 phrases."
response = query_llm(prompt)
print(f"Prompt: {prompt}\n\nRéponse: {response}")

## 4. Exploration des paramètres

Les modèles de langage sont influencés par plusieurs paramètres qui contrôlent la génération de texte. Explorons les deux principaux:

### Temperature
- Contrôle la "créativité" ou l'aléatoire des réponses
- Valeurs basses (0.0-0.3): Réponses plus déterministes, cohérentes
- Valeurs moyennes (0.4-0.7): Bon équilibre
- Valeurs élevées (0.8-1.0): Réponses plus créatives, variées, parfois incohérentes

### Max Tokens
- Limite la longueur de la réponse générée
- Un token représente environ 4 caractères en anglais
- Une page de texte = environ 500-700 tokens

In [ ]:
# Testons différentes températures pour la même question
prompt = "Donne-moi une idée de startup innovante"

print("Temperature = 0.1 (Conservateur):")
response_conservative = query_llm(prompt, temperature=0.1)
print(response_conservative)
print("\n" + "-"*50 + "\n")

print("Temperature = 0.7 (Équilibré):")
response_balanced = query_llm(prompt, temperature=0.7)
print(response_balanced)
print("\n" + "-"*50 + "\n")

print("Temperature = 1.0 (Créatif):")
response_creative = query_llm(prompt, temperature=1.0)
print(response_creative)

In [ ]:
# Testons différentes limites de tokens
prompt = "Explique en détail comment fonctionne un modèle de langage comme GPT"

print("Max Tokens = 50 (Court):")
response_short = query_llm(prompt, max_tokens=50)
print(response_short)
print("\n" + "-"*50 + "\n")

print("Max Tokens = 200 (Moyen):")
response_medium = query_llm(prompt, max_tokens=200)
print(response_medium)
print("\n" + "-"*50 + "\n")

print("Max Tokens = 500 (Long):")
response_long = query_llm(prompt, max_tokens=500)
print(response_long)

## 5. Exercice pratique

Maintenant, c'est à votre tour! Complétez l'exercice suivant pour pratiquer l'utilisation du SLM:


### Exercice : Créer une fonction d'assistant spécialisé
Complétez la fonction ci-dessous pour créer un assistant spécialisé dans un domaine particulier.

In [ ]:
def create_specialized_assistant(domain):
    """
    Crée un assistant spécialisé dans un domaine particulier.
    
    Args:
        domain (str): Le domaine de spécialisation (ex: "programmation", "marketing", "science")
        
    Returns:
        function: Une fonction qui prend une question et retourne une réponse de l'assistant
    """
    # À COMPLÉTER: Définir un prompt système approprié selon le domaine
    system_prompts = {
        "programmation": "Tu es un expert en programmation informatique.",
        # Ajoutez d'autres domaines ici
    }
    
    # À COMPLÉTER: Récupérer le prompt système approprié ou utiliser un par défaut
    
    def ask(question):
        # À COMPLÉTER: Construire le prompt complet et interroger le modèle
        pass
    
    return ask

# Test de votre implémentation
# Décommentez ces lignes une fois votre fonction complétée
# programming_assistant = create_specialized_assistant("programmation")
# response = programming_assistant("Explique ce qu'est une fonction récursive et donne un exemple simple.")
# print(response)

## 7. Conclusion et ressources supplémentaires

Dans ce notebook, nous avons:
- Compris les différences entre LLMs et SLMs
- Appris à interagir avec un SLM local via l'API OpenAI
- Exploré les paramètres qui influencent les réponses
- Créé des fonctions spécialisées utilisant le SLM

### Ressources supplémentaires:
- [Guide de prompting OpenAI](https://platform.openai.com/docs/guides/prompt-engineering)
- [Documentation llama.cpp](https://github.com/ggerganov/llama.cpp)
- [Hugging Face - Modèles SLM populaires](https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads)
- [Learn Prompting](https://learnprompting.org/) - Guide complet sur l'art du prompting
